In [56]:
#imports
import os
import re

import pandas as pd
import tkinter as tk

from datetime import datetime
from smartapi import SmartConnect
from symbol_selector import UserInterface


# get the master DF from json
tokens_url = "https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json"
df_master = pd.read_json(tokens_url)
s_map = pd.Series(df_master.name.values,index=df_master.symbol).to_dict()
s_concat_list = ["{}  ||  {}".format(key, value) for key, value in s_map.items()]
print("--> DF Master read successful")
# print(df_master.head())


# Api connection
uname = "SBD1199"
pwd = "Pass2406@ab"
api_key = "CGz5s9Tv"
api_key_feed = "bgBHbiTo"
secret = "1c873097-b4f6-4b50-ae5f-5b70341a0347"

try:
    obj = SmartConnect(api_key=api_key)
    data = obj.generateSession(uname, pwd)
    refresh_token = data['data']['refreshToken']
    feedtoken = obj.getfeedToken()
    user_profile = obj.getProfile(refresh_token)
    print("--> API connection successful")
#     print(user_profile)
except Exception as e:
    print("API connection Failed: {}".format(str(e)))

    
def get_historic_data(exchange, symbol, token, interval, fdate, todate):
    try:
        params={
            "exchange": exchange,
            "tradingsymbol": symbol,
            "symboltoken": token,
            "interval": interval,
            "fromdate": fdate,
            "todate": todate
        }
        candle_data = obj.getCandleData(params)['data']
        df_candle = pd.DataFrame(candle_data)
        if df_candle.empty:
            return None
        
        df_candle.insert(0, 'Symbol', each_symbol)
        df_candle = df_candle.rename(
                    columns = {0: "Datetime", 1: "Open", 2: "High", 3: "Low", 4: "Close", 5: "Volume"}
                   )        
        df_candle['Datetime'] = pd.to_datetime(df_candle['Datetime'])
        df_candle = df_candle.set_index('Symbol')
        return df_candle
    
    except Exception as e:
        print("Something went wrong - {}".format(e))
        return None

    
def get_expiry_data(index,expiry,start_strike,increment, end_strike):
    option_list = ['CE', 'PE']
    l_range = int((end_strike-start_strike)/increment)
    l_range = l_range+1
    derived_symbols = list()
    for i in range(0, l_range):
        for j in option_list:
            end_str = str(start_strike + (i*increment)) + j
            derived_symbols.append(index + expiry + end_str)
    return derived_symbols

--> DF Master read successful
--> API connection successful


In [57]:
import time


index='NIFTY'
expiry ='23SEP21'

derived_symbols = get_expiry_data(index=index, 
                                  expiry=expiry, 
                                  start_strike=14150, 
                                  increment=50, 
                                  end_strike=19250)

exchange = 'NFO'
timeframe = "ONE_MINUTE"
from_date = "2021-08-01 00:00"
to_date = "2021-09-23 23:59"


# parent_dir = "C:\\Users\\haris\\Desktop\\Autodha"
parent_dir = os.getcwd()
directory = index+expiry
path = os.path.join(parent_dir, directory)
c_path = os.path.join(path, 'CE')
p_path = os.path.join(path, 'PE')

os.makedirs(path, exist_ok=True)
os.makedirs(c_path, exist_ok=True)
os.makedirs(p_path, exist_ok=True)

run_count = 0
pass_count = 0

# NIFTY23SEP2114250CE 47155

for enum, each_symbol in enumerate(derived_symbols):
    print("\n----------------------------------------------------------")
    print("{}. Getting historic data for {}".format(enum, each_symbol))
    run_count+=1
    ser = df_master.loc[df_master['symbol'] == each_symbol]
    if ser.empty:
        print("symbol {} doesn't exist in the api json".format(each_symbol))
        continue
    print("Token found: {}".format(each_token))
    each_token = str(ser['token'].to_list()[0])
    daily_data = get_historic_data(exchange, each_symbol, each_token, timeframe, from_date, to_date)
    result_df = pd.DataFrame(daily_data)
    curr_time = datetime.now()
    ctime = curr_time.strftime("%Y-%m-%d_%H-%M-%S") 
    
    if result_df.empty:
        print("Data for the symbol doesn't exist")
    else:
        if each_symbol.endswith('CE'):
            result_df.to_csv("{}\\{}_{}.csv".format(c_path, each_symbol, ctime), index=True)
            print("csv file has been created for {} at {}".format(each_symbol, ctime))
            pass_count+=1

        if each_symbol.endswith('PE'):
            result_df.to_csv("{}\\{}_{}.csv".format(p_path, each_symbol, ctime), index=True)
            print("csv file has been created for {} at {}".format(each_symbol, ctime))
            pass_count+=1

        
    time.sleep(1)
print("Total Symbols: {}, csvs downloaded: {}".format(run_count, pass_count))
        


----------------------------------------------------------
0. Getting historic data for NIFTY23SEP2114150CE
symbol NIFTY23SEP2114150CE doesn't exist in the api json

----------------------------------------------------------
1. Getting historic data for NIFTY23SEP2114150PE
symbol NIFTY23SEP2114150PE doesn't exist in the api json

----------------------------------------------------------
2. Getting historic data for NIFTY23SEP2114200CE
Token found: 49643
Data for the symbol doesn't exist

----------------------------------------------------------
3. Getting historic data for NIFTY23SEP2114200PE
Token found: 50807
                                     Datetime  Open  High  Low  Close  Volume
Symbol                                                                       
NIFTY23SEP2114200PE 2021-09-13 12:47:00+05:30   2.0   2.0  2.0    2.0       1
NIFTY23SEP2114200PE 2021-09-13 12:50:00+05:30   2.0   2.0  2.0    2.0       0
csv file has been created for NIFTY23SEP2114200PE at 2021-09-19_18


----------------------------------------------------------
27. Getting historic data for NIFTY23SEP2114800PE
Token found: 49602
                                     Datetime  Open  High   Low  Close  Volume
Symbol                                                                        
NIFTY23SEP2114800PE 2021-08-26 12:43:00+05:30  25.0  25.0  25.0   25.0       1
NIFTY23SEP2114800PE 2021-08-26 12:44:00+05:30  25.0  25.0  10.0   10.0       1
csv file has been created for NIFTY23SEP2114800PE at 2021-09-19_18-26-43

----------------------------------------------------------
28. Getting historic data for NIFTY23SEP2114850CE
Token found: 49603
Data for the symbol doesn't exist

----------------------------------------------------------
29. Getting historic data for NIFTY23SEP2114850PE
Token found: 49604
                                     Datetime  Open  High   Low  Close  Volume
Symbol                                                                        
NIFTY23SEP2114850PE 2021-09-15 1


----------------------------------------------------------
52. Getting historic data for NIFTY23SEP2115450CE
Token found: 49652
Data for the symbol doesn't exist

----------------------------------------------------------
53. Getting historic data for NIFTY23SEP2115450PE
Token found: 49653
                                     Datetime  Open  High  Low  Close  Volume
Symbol                                                                       
NIFTY23SEP2115450PE 2021-09-17 14:27:00+05:30   1.7   1.7  1.7    1.7       1
NIFTY23SEP2115450PE 2021-09-17 14:28:00+05:30   1.7   1.7  1.7    1.7       0
csv file has been created for NIFTY23SEP2115450PE at 2021-09-19_18-27-14

----------------------------------------------------------
54. Getting historic data for NIFTY23SEP2115500CE
Token found: 49654
                                     Datetime    Open    High      Low  \
Symbol                                                                   
NIFTY23SEP2115500CE 2021-09-17 09:16:00+05:30 

csv file has been created for NIFTY23SEP2115800CE at 2021-09-19_18-27-30

----------------------------------------------------------
67. Getting historic data for NIFTY23SEP2115800PE
Token found: 49689
                                     Datetime   Open   High    Low  Close  \
Symbol                                                                      
NIFTY23SEP2115800PE 2021-08-26 15:06:00+05:30  15.55  15.55  15.55  15.55   
NIFTY23SEP2115800PE 2021-08-26 15:07:00+05:30  15.55  15.55  15.55  15.55   

                     Volume  
Symbol                       
NIFTY23SEP2115800PE       1  
NIFTY23SEP2115800PE       0  
csv file has been created for NIFTY23SEP2115800PE at 2021-09-19_18-27-31

----------------------------------------------------------
68. Getting historic data for NIFTY23SEP2115850CE
Token found: 49690
                                     Datetime     Open     High      Low  \
Symbol                                                                     
NIFTY23SEP21158


----------------------------------------------------------
81. Getting historic data for NIFTY23SEP2116150PE
Token found: 49711
                                     Datetime  Open  High  Low  Close  Volume
Symbol                                                                       
NIFTY23SEP2116150PE 2021-09-16 12:05:00+05:30   4.5   4.5  4.5    4.5       4
NIFTY23SEP2116150PE 2021-09-16 12:06:00+05:30   4.5   4.5  4.5    4.5       0
csv file has been created for NIFTY23SEP2116150PE at 2021-09-19_18-27-49

----------------------------------------------------------
82. Getting historic data for NIFTY23SEP2116200CE
Token found: 49712
                                     Datetime   Open   High    Low  Close  \
Symbol                                                                      
NIFTY23SEP2116200CE 2021-08-27 09:53:00+05:30  495.4  495.4  495.4  495.4   
NIFTY23SEP2116200CE 2021-08-27 09:54:00+05:30  495.4  495.4  495.4  495.4   

                     Volume  
Symbol            

...


----------------------------------------------------------
122. Getting historic data for NIFTY23SEP2117200CE
Token found: 49792
                                     Datetime  Open  High   Low  Close  Volume
Symbol                                                                        
NIFTY23SEP2117200CE 2021-08-30 10:36:00+05:30  60.0  60.0  60.0   60.0       1
NIFTY23SEP2117200CE 2021-08-30 10:37:00+05:30  60.0  60.0  60.0   60.0       0
csv file has been created for NIFTY23SEP2117200CE at 2021-09-19_18-28-48

----------------------------------------------------------
123. Getting historic data for NIFTY23SEP2117200PE
Token found: 49794
                                     Datetime   Open   High    Low  Close  \
Symbol                                                                      
NIFTY23SEP2117200PE 2021-08-31 15:21:00+05:30  258.5  258.5  258.5  258.5   
NIFTY23SEP2117200PE 2021-08-31 15:22:00+05:30  258.5  258.5  258.5  258.5   

                     Volume  
Symbol      


----------------------------------------------------------
136. Getting historic data for NIFTY23SEP2117550CE
Token found: 37793
                                     Datetime  Open  High   Low  Close  Volume
Symbol                                                                        
NIFTY23SEP2117550CE 2021-09-02 13:04:00+05:30  60.0  60.0  60.0   60.0       1
NIFTY23SEP2117550CE 2021-09-02 13:05:00+05:30  60.0  60.0  60.0   60.0       0
csv file has been created for NIFTY23SEP2117550CE at 2021-09-19_18-29-09

----------------------------------------------------------
137. Getting historic data for NIFTY23SEP2117550PE
Token found: 37803
                                     Datetime   Open   High    Low  Close  \
Symbol                                                                      
NIFTY23SEP2117550PE 2021-09-03 11:13:00+05:30  382.0  382.0  382.0  382.0   
NIFTY23SEP2117550PE 2021-09-03 11:14:00+05:30  382.0  382.0  382.0  382.0   

                     Volume  
Symbol      


----------------------------------------------------------
150. Getting historic data for NIFTY23SEP2117900CE
Token found: 44472
                                     Datetime  Open  High   Low  Close  Volume
Symbol                                                                        
NIFTY23SEP2117900CE 2021-09-01 14:51:00+05:30  20.2  20.2  20.2   20.2       2
NIFTY23SEP2117900CE 2021-09-01 14:52:00+05:30  20.2  20.2  20.2   20.2       0
csv file has been created for NIFTY23SEP2117900CE at 2021-09-19_18-29-28

----------------------------------------------------------
151. Getting historic data for NIFTY23SEP2117900PE
Token found: 43240
                                     Datetime    Open    High     Low   Close  \
Symbol                                                                          
NIFTY23SEP2117900PE 2021-09-03 13:36:00+05:30  629.45  629.45  629.45  629.45   
NIFTY23SEP2117900PE 2021-09-03 13:37:00+05:30  629.45  629.45  629.45  629.45   

                     Volum


----------------------------------------------------------
164. Getting historic data for NIFTY23SEP2118250CE
Token found: 35048
                                     Datetime  Open  High   Low  Close  Volume
Symbol                                                                        
NIFTY23SEP2118250CE 2021-09-09 15:28:00+05:30  3.25  3.25  3.25   3.25       1
NIFTY23SEP2118250CE 2021-09-09 15:29:00+05:30  3.25  3.25  3.25   3.25       0
csv file has been created for NIFTY23SEP2118250CE at 2021-09-19_18-29-47

----------------------------------------------------------
165. Getting historic data for NIFTY23SEP2118250PE
Token found: 36499
                                     Datetime   Open   High    Low  Close  \
Symbol                                                                      
NIFTY23SEP2118250PE 2021-09-09 15:28:00+05:30  863.9  863.9  863.9  863.9   
NIFTY23SEP2118250PE 2021-09-09 15:29:00+05:30  863.9  863.9  863.9  863.9   

                     Volume  
Symbol      

csv file has been created for NIFTY23SEP2118600PE at 2021-09-19_18-30-05

----------------------------------------------------------
180. Getting historic data for NIFTY23SEP2118650CE
Token found: 38625
                                     Datetime  Open  High   Low  Close  Volume
Symbol                                                                        
NIFTY23SEP2118650CE 2021-09-16 14:58:00+05:30  1.85  1.85  1.85   1.85       1
NIFTY23SEP2118650CE 2021-09-16 14:59:00+05:30  1.85  1.85  1.85   1.85       0
csv file has been created for NIFTY23SEP2118650CE at 2021-09-19_18-30-06

----------------------------------------------------------
181. Getting historic data for NIFTY23SEP2118650PE
Token found: 38626
Data for the symbol doesn't exist

----------------------------------------------------------
182. Getting historic data for NIFTY23SEP2118700CE
Token found: 38627
                                     Datetime  Open  High  Low  Close  Volume
Symbol                              